In [40]:
%load_ext micropython_magic
%reload_ext micropython_magic

The micropython_magic extension is already loaded. To reload it, use:
  %reload_ext micropython_magic


In [41]:
%mpy -s {"/dev/cu.usbmodem1234561"}

In [42]:
# %%micropython

import time
from config import config
from secrets import secrets
from machine import RTC
from mp_libs.network import Network
from mp_libs.protocols import espnow_protocol
from mp_libs.protocols import min_iot_protocol
from mp_libs.protocols.wifi_protocols import WifiProtocol
from mp_libs.time import ptp

TIMEOUT_MSEC = 5000

rtc = RTC()
wifi = Network.create_wifi()
miniot_network = Network.create_min_iot("GTY")
epn = miniot_network.transport.transport.transport

wifi.connect()
miniot_network.connect()
wifi.ntp_time_sync()

print(f"Initial RTC datetime: {rtc.datetime()}")
print(f"Initial RTC now: {rtc.now()}")
print(f"My MAC: {epn.wifi._sta.config('mac')}")
print(f"EPN Peer: {config['epn_peer_mac']}")
print(f"Channel: {epn.wifi._sta.config('channel')}")
print(f"EPN timeout: {epn.epn.config('timeout_ms')}")

while True:
    num_sync_cycles = 1
    rxed_packets = []
    data_available = False

    # Receive data
    while not data_available:
        data_available = miniot_network.receive(rxed_packets)
        time.sleep_ms(100)

    # Parse rx'ed packets
    for packet in rxed_packets:
        if ptp.is_ptp_msg(packet.msg):
            ptp_type, payload = ptp.parse_msg(packet.msg)
        else:
            print(f"Rx'ed unexpected message: {packet}")
            continue

        # Perform PTP sync
        print(f"ptp_type: {ptp_type}, payload: {payload}")
        if ptp_type == ptp.PtpMsgs.SYNC_REQ:
            ptp.sequence_master(
                miniot_network,
                lambda miniot_msg: miniot_msg.msg,
                TIMEOUT_MSEC,
                num_sync_cycles=payload)

            print(f"now: {rtc.datetime()}")


2024-09-29 20:37:56.466856 INFO-wifi-protocols:Disconnecting wifi...
2024-09-29 20:37:56.471469 WARNING-espnow-protocol:Peer has already been added, skipping. Peers: b'p\x04\x1d\xad|\xc0'
2024-09-29 20:37:56.473217 DEBUG-espnow-protocol:espnow configured
2024-09-29 20:37:56.475208 INFO-wifi-protocols:Connecting wifi...
2024-09-29 20:37:56.477470 DEBUG-wifi-protocols:Connecting to AP: D&A_Home
2024-09-29 20:37:58.19937 INFO-wifi-protocols:Wifi is connected: ('192.168.0.138', '255.255.255.0', '192.168.0.1', '192.168.0.1')
2024-09-29 20:37:58.22042 INFO-miniot:Connecting MinIoT...
2024-09-29 20:37:58.23913 INFO-espnow-protocol:Connecting espnow...
Initial RTC datetime: (2024, 9, 29, 6, 20, 37, 58, 102)
Initial RTC now: 780957478000806
My MAC: b"H'\xe2\xc7n\xfc"
EPN Peer: b'p\x04\x1d\xad|\xc0'
Channel: 1
EPN timeout: 10000
2024-09-29 20:38:02.314705 DEBUG-espnow-protocol:Processing packet: <EspnowPacket object at 3c191570>
2024-09-29 20:38:02.316650 DEBUG-espnow-protocol:Sending SCAN RESP
